<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/MovieSimil_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [10]:
import pandas as pd
data = pd.read_csv("/gdrive/My Drive/RogerHeederer/NLP_entry/data/movies_metadata.csv", low_memory=False)
data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [11]:
data.shape

(45466, 24)

In [12]:
# 영화제목 : title,  줄거리 : overview
# title과 overview 컬럼에 대해 유사도를 구해보고자 한다

# 데이터 양 줄이고 시작해보겠다
data = data.head(20000)

In [13]:
data['overview'].isnull().sum()

135

In [14]:
data['overview'] = data['overview'].fillna('') #overview 컬럼에서 null을 제거

In [17]:
data['overview'].isnull().sum()

0

In [46]:
pd.DataFrame(data['overview'])[0:5]

,overview
0,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...


In [47]:
pd.DataFrame(data['title'])[0:5]

,title
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II


In [18]:
#Overview 열에 tf-idf 수행
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

# 2만개의 영화에 대해, 47487 단어가 쓰였음을 확인

(20000, 47487)


In [22]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
cosine_sim.shape

(20000, 20000)

In [53]:
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [35]:
data.index

RangeIndex(start=0, stop=20000, step=1)

In [24]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [26]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [59]:
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = indices[title] # 선택한 영화의 타이틀로부터 해당되는 인덱스 받아오기
  sim_scores = list(enumerate(cosine_sim[idx])) #모든 영화에 대해서 해당 영화 유사도 구하기
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #유사도 기준으로 정렬
  sim_scores = sim_scores[1:11] #가장 유사한 10개 받아오기
  movie_indices = [i[0] for i in sim_scores] #유사한 영화 인덱스 받아오기
  return data['title'].iloc[movie_indices]


In [60]:
get_recommendations('The Dark Knight Rises')

[(14449, 0.059881491005434696), (7238, 0.059518100773835445), (1558, 0.059430219796551005), (1901, 0.059300486837714106), (6651, 0.059208933504148444), (15472, 0.058702309709302486), (3282, 0.058385022745255304), (16888, 0.05829260329855165), (18531, 0.05819904771184631), (11930, 0.05798499202407875), (2274, 0.05784444825079399), (10237, 0.05783410459578986), (4076, 0.05778707932115698), (3613, 0.05777414916972472), (2878, 0.05777394121025318), (16103, 0.05775967734001844), (12887, 0.05773205231015999), (12299, 0.057717979272778386), (16950, 0.05770420914490712), (18421, 0.05768865302564154)]


12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object